In [1]:
!pip3 install datasets
!pip3 install ipywidgets widgetsnbextension pandas-profiling
!pip3 install tqdm
!pip3 install transformers
!pip3 install tensorflow
!pip3 install torch
!pip3 install accelerate
!pip3 install evaluate
!pip3 install tf-keras
!pip3 install peft
!pip3 install -U bitsandbytes

In [2]:
!pip3 install -q --no-deps xformers trl bitsandbytes

!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip3 install intel-extension-for-pytorch
!pip3 install oneccl_bind_pt --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/

Looking in indexes: https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://pytorch-extension.intel.com/release-whl/stable/cpu/us/


In [5]:
!pip install --force-reinstall 'https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/continuous-release_multi-backend-refactor/bitsandbytes-0.44.1.dev0-py3-none-manylinux_2_24_x86_64.whl'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
  Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import intel_extension_for_pytorch as ipex
import torch

quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    quantization_config=quantization_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


2025-04-08 11:03:20.713547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744103000.774452   66317 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744103000.792561   66317 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744103000.925235   66317 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744103000.925256   66317 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744103000.925257   66317 computation_placer.cc:177] computation placer alr

In [2]:

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)


KeyboardInterrupt



In [2]:
from datasets import load_dataset

dataset = load_dataset("nyu-mll/glue", "mnli", split='train')
dataset = dataset.train_test_split(test_size=0.1)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 353431
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 39271
    })
})


In [3]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM


small_train = dataset["train"].shuffle(seed=42).select(range(50))
small_eval = dataset["test"].shuffle(seed=42).select(range(20))
def get_label(label):
    if label == 0:
        word = "True"
    elif label == 1:
        word = "Neutral"
    else:
        word = "False"
    return word
    
def formatting_train_func(data):
    prompt = f"{data["premise"]} \nQuestion: {data["hypothesis"]} \nTrue, False or Neither?\nAnswer:"
    label = get_label(data["label"])

    messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": label}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return {"text": text}

def formatting_eval_func(data):
    prompt = f"{data['premise']} \nQuestion: {data['hypothesis']} \nTrue, False or Neither?\nAnswer:"
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return {
        "text": text,
        "label": data["label"] # store this so we can compare predictions later
    }


small_train = small_train.map(formatting_train_func, remove_columns=["hypothesis", "premise", "label"])
small_eval = small_eval.map(formatting_eval_func, remove_columns=["hypothesis", "premise", "label"])

print(small_train[0])
print(small_eval[0])
response_template = "<|im_start|>assistant\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

{'idx': 78472, 'text': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nbecause it is a matter of of losing your job if you get caught say a second time \nQuestion: It is not worth losing your job over. \nTrue, False or Neither?\nAnswer:<|im_end|>\n<|im_start|>assistant\nNeutral<|im_end|>\n<|im_start|>assistant\n'}
{'label': 2, 'idx': 21046, 'text': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nIn addition, recent studies have shown a relationship between PM and non-fatal heart attacks, which suggests that some of the deaths due to PM may be due to fatal heart attacks (Peters et al. \nQuestion: PM does not affect the heart. \nTrue, False or Neither?\nAnswer:<|im_end|>\n<|im_start|>assistant\n'}


In [4]:
def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=1024)

small_train = small_train.map(tokenize)
small_eval = small_eval.map(tokenize)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [5]:
from peft import LoraConfig, TaskType, get_peft_model


# create LoRA configuration object
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, # type of task to train on
    inference_mode=False, # set to False for training
    r=8, # dimension of the smaller matrices
    lora_alpha=32, # scaling factor
    lora_dropout=0.1 # dropout of LoRA layers
)

model.add_adapter(lora_config, adapter_name="lora_1")

In [ ]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
import evaluate

from trl import SFTConfig

sft_config = SFTConfig(
    output_dir="mnli_test",
    num_train_epochs=1,
    logging_dir="./logs",
    save_strategy="epoch",
    push_to_hub=False,
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=small_train,
    eval_dataset=small_eval,
    args=sft_config,
    data_collator=collator,
)

trainer.train()

In [ ]:

metrics = trainer.evaluate()
print(metrics)
